In [17]:
base_link = 'https://www.poetryfoundation.org/search?query='     #base_link + 'genre of poem'
meaning_base_link = 'https://www.momjunction.com/baby-names/'
from bs4 import BeautifulSoup as bs
import requests
import random
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
from time import sleep
from selenium import webdriver
from lxml import etree
import pandas as pd

In [18]:
def add_products(main):
    f=open('products.html','w')
        # HTML STARTS HERE
    message_1 = """

    <html>

    <head>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" integrity="sha384-ggOyR0iXCbMQv3Xipma34MD+dH/1fQ784/j6cY/iJTQUOhcWr7x9JvoRxT2MZw1T" crossorigin="anonymous">
    <script src="https://code.jquery.com/jquery-3.3.1.slim.min.js" integrity="sha384-q8i/X+965DzO0rT7abK41JStQIAqVgRVzpbzo5smXKp4YfRvH+8abtTE1Pi6jizo" crossorigin="anonymous"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.14.7/umd/popper.min.js" integrity="sha384-UO2eT0CpHqdSJQ6hJty5KVphtPhzWj9WO1clHTMGa3JDZwrnQq4sF86dIHNDz0W1" crossorigin="anonymous"></script>
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/js/bootstrap.min.js" integrity="sha384-JjSmVgyd0p3pXB1rRibZUAYoIIy6OrQ6VrjIEaFf/nJGzIxFDsf4x0xIM+B07jRM" crossorigin="anonymous"></script> 

    </head>

    <body>
      <table class="table table-striped table-bordered">
      <thead class="thead-dark">
         <tr>
            <th>Product Name</th>
            <th>True Price</th>
            <th>Sale Price</th>
            <th>Rating</th>
         </tr>
         </thead>
         """
    str_=""
    for i in main:
        str_+="<tr>"+"<td>" + str(i[0]) + "</td>"
        str_+="<td>" + str(i[1])+ "</td>"
        str_+="<td>" + str(i[2])+ "</td>"
        str_+="<td>" + str(i[3])+ "</td>"
        
    message_2="""
          </table>
       </body>
    </html>
    
    """
    message=message_1+str_+message_2
    f.write(message)
    f.close()

In [19]:
name = input('Hi, what is your name? [answer in one word]   Your reply -> ')

name_meaning_link = meaning_base_link + name +'/'

list_no_tags = []

text = requests.get(name_meaning_link).text         #requesting for page
soup = bs(text,'lxml')              
s = soup.find_all(['label', 'span'])                #finding all the codes with the given tags

for i in range(len(s)):
    try:
        s1 = str(s[i]).split('>')
        s2 = s1[1].split('<')
        list_no_tags.append(s2[0])
    except:
        pass
    
def meaning(list_no_tags):
    for i in range(len(list_no_tags)):
        if 'Meaning :' in list_no_tags[i]:
                m = list_no_tags[i+1]
                return m    
first_name = ['What a name!!', 'Nice name.', 'I like your name. ', 'I wish i had the same name.']
try:
    print(random.choice(first_name) + ' Do you know the meaning of your name is ' + meaning(list_no_tags))  
except:
    print(random.choice(first_name))


feeling = input('Okay ' + name.upper() + '!' + 'How are you feeling today?' +'  '+ name.upper() +  ' -> ')
feels = ""
s = ""
lemmatizer = WordNetLemmatizer()
for k in feeling.split(" "):
    s = s+ " " + lemmatizer.lemmatize(k, 'v')
blob = TextBlob(s)
sentence = blob.sentences[0]
for word, pos in sentence.tags:      # word --> word itself, pos --> part of speech
    if pos == 'JJ' or pos== 'NN':
        feels = word

poem_link = base_link +  feels   + '&refinement=poems'


print('I hope you will like this poem: ')
print('\n')


all_links = []         #contains the link of the poems based on how user is feeling
req_links = []

text = requests.get(poem_link).text
soup = bs(text, 'lxml')
s = soup.find_all('a')
for i in s:
    if (feels and '/poems/') in str(i) and 'browse#' not in str(i) and '/guides' not in str(i) and '/poem-of-the-day' not in str(i):
        req_links.append(i)
for link in req_links:
     all_links.append(link.get('href'))
    
text = requests.get(all_links[random.choice(range(len(all_links)))]).text
soup = bs(text, 'lxml')
for line in soup.find_all(style="text-indent: -1em; padding-left: 1em;"):         #extracting text with given style tag
    print(line.text)        
    
its = []     #its --> item to search

print(name.upper() + ', do you want to shop online?')
response_1 = input()
sleep(1)
if 'yes' in response_1.lower():
    print('Okay then, what would you want me to search for you?')
    resp_search = input()
    blob = TextBlob(resp_search)
    sentence = blob.sentences[0]
    for word, pos in sentence.tags:      # word --> word itself, pos --> part of speech
        if pos == 'NN' or pos == 'NNS':
            its.append(word)

#----------------------Amazon search-----------------------------------------#

query = "".join(its)
main = [] 

driver = webdriver.Chrome('/Users/vaibhav/Downloads/chromedriver-2')
driver.get('https://www.amazon.in')
sleep(1)

searchbox = driver.find_element_by_xpath('//*[@id="twotabsearchtextbox"]')
searchbox.send_keys(query)
sleep(1)

button = driver.find_element_by_xpath('//*[@id="nav-search"]/form/div[2]/div/input')
button.click()

for j in range(1, 5):       #number of times it goes to next page(crawling)

    #------------------------------#
    soup = bs(driver.page_source, 'lxml')
    parser = etree.HTMLParser()
    tree = etree.fromstring(str(soup), parser)
    #------------------------------#

    for i in range(20):    #number of products scraped(scraping)
        temp = []
        item_number = '//div[@data-index = "' + str(i) + '"]'

        try:
            item_name = tree.xpath(item_number + '//span[contains(@class, "a-size-medium a-color-base a-text-normal")]/text()')[0]
        except: 
            item_name = "N/A"
        try:
            rating = tree.xpath(item_number + '//span[contains(@class, "a-icon-alt")]/text()')[0]
        except:
            rating = "N/A"
        try:
            true_price = tree.xpath(item_number + '//span[contains(@class, "a-offscreen")]/text()')[1]
            sale_price = tree.xpath(item_number + '//span[contains(@class, "a-offscreen")]/text()')[0]
        except:
            true_price = "N/A"
            sale_price = "N/A"

        temp.append(item_name)
        temp.append(true_price)
        if len(sale_price)<15:
            temp.append(sale_price)
        else:
            temp.append("N/A")
        temp.append(rating)

        main.append(temp)
        
    driver.get("https://www.amazon.in/s?k=" + str(query) + "&page=" + str(j+1) + "&qid=1579856088&ref=sr_pg_"+str(j))
        
sleep(1)

driver.quit()

add_products(main)

print("So" + name + ", to view " + query + " click on the link given below.")
print("http://localhost:8888/view/products.html")

Hi, what is your name? [answer in one word]   Your reply -> vaibhav
I like your name.  Do you know the meaning of your name is One who is fortunate, prosperous and intelligent.
Okay VAIBHAV!How are you feeling today?  VAIBHAV -> I am feeling good
I hope you will like this poem: 


January. I pluck it,
this feather flapping in the high mesquite
only head-high, caught by the down,
iridescent, turkey. Another
feather hugging the ditch
along the fence line and another...
A coyote somewhere naps 
happy, grinning like the feather
evolved from a leaf. What luck. 

Clouds lift above the field
as if to swallow my eye
into hunger. Good hunger.
The greatest eye must
behold me like an ember
dropped into a finch nest,
and I smoke at the mouth
like a gun dreaming in a safe
of a war it can win by virtue
of its praise. I have lost
the killer phrase I concocted
on my country walk
with the feather in my pocket.
I cock it.
VAIBHAV, do you want to shop online?
yes I would love to
Okay then, what would you